# Play with first models

In [57]:
import pandas as pd
#get data:
file_features = 'dengue_features_train.csv'
file_labels = 'dengue_labels_train.csv'

df_features = pd.read_csv("../data/"+file_features)
df_labels = pd.read_csv("../data/"+file_labels)

#split data initially for train-validation and TEST:
from sklearn.model_selection import train_test_split
X = df_features
y = df_labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Begin transformation pipeline:

In [58]:
#fix date, drop extras
X_train.week_start_date = pd.to_datetime(X_train.week_start_date, yearfirst=True)
X_train.drop(['year', 'weekofyear'], axis=1, inplace=True)
y_train.drop(['city', 'year', 'weekofyear'], axis=1, inplace=True)

/home/ant_on_su/miniconda3/lib/python3.5/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/ant_on_su/miniconda3/lib/python3.5/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [59]:
#fill NANs
X_train.fillna(method='ffill', inplace=True)

/home/ant_on_su/miniconda3/lib/python3.5/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [60]:
#One-hot ecode 'city'
X_train = pd.get_dummies(X_train)

In [61]:
X_train.head()

,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,...,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,city_iq,city_sj
546,2000-10-28,0.15350,0.229800,0.148880,0.091525,62.57,300.018571,299.971429,295.725714,302.6,...,62.57,17.117143,3.457143,27.142857,6.528571,31.1,23.3,24.1,0,1
877,2007-03-12,0.03730,0.001800,0.164757,0.175486,0.00,299.015714,299.335714,294.020000,301.0,...,0.00,15.382857,2.985714,26.714286,8.042857,31.7,21.7,0.3,0,1
202,1994-03-19,0.14680,-0.048150,0.194683,0.140550,0.00,297.501429,297.671429,292.864286,299.8,...,0.00,14.334286,2.571429,25.842857,7.957143,31.1,21.1,16.8,0,1
1342,2008-04-22,0.27060,0.371314,0.344229,0.283071,42.05,296.760000,297.414286,296.382857,301.9,...,42.05,17.964286,4.885714,26.650000,7.700000,33.0,21.6,85.9,1,0
587,2001-08-13,0.08985,0.127833,0.200814,0.205114,29.00,300.742857,300.857143,297.380000,302.9,...,29.00,18.892857,2.600000,28.642857,5.585714,32.2,25.6,24.3,0,1
